# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bucerias,20.7500,-105.3333,28.99,84,75,2.57,MX,1728610203
1,1,karratha,-20.7377,116.8463,31.70,37,0,7.15,AU,1728610204
2,2,napili-honokowai,20.9708,-156.6705,30.64,83,100,5.78,US,1728610205
3,3,papatowai,-46.5619,169.4708,13.37,60,72,3.84,NZ,1728610206
4,4,mariinsk,56.2139,87.7472,-2.62,62,100,3.56,RU,1728610207


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size='Humidity',
    color='City',
    hover_cols=['City', 'Humidity'],
    tiles='OSM',
    title='City Humidity Levels'
)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
cleaned_cities = filtered_cities.dropna()

# Display sample data
cleaned_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,tabas,33.5959,56.9244,22.37,18,0,2.05,IR,1728610268
60,60,holualoa,19.6228,-155.9522,26.25,75,0,3.60,US,1728610271
67,67,hilo,19.7297,-155.0900,26.85,84,0,3.60,US,1728610030
71,71,qina,26.1642,32.7267,24.54,36,0,2.01,EG,1728610285
84,84,at taraf,25.3628,49.7250,26.30,65,0,2.57,SA,1728610300


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
57,tabas,IR,33.5959,56.9244,18,
60,holualoa,US,19.6228,-155.9522,75,
67,hilo,US,19.7297,-155.0900,84,
71,qina,EG,26.1642,32.7267,36,
84,at taraf,SA,25.3628,49.7250,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'radius': radius,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tabas - nearest hotel: هتل امیر
holualoa - nearest hotel: Kona Hotel
hilo - nearest hotel: Dolphin Bay Hotel
qina - nearest hotel: Basma Hotel
at taraf - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
north platte - nearest hotel: Howard Johnson Inn - North Platte
figuig (centre) - nearest hotel: No hotel found
jining - nearest hotel: No hotel found
lee acres - nearest hotel: Sarabande Bed & Breakfast
grantsville - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
klamath falls - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
silver city - nearest hotel: Murray Hotel
port isabel - nearest hotel: Queen Isabel Inn
conde - nearest hotel: Paladino
nelson bay - nearest hotel: No hotel found
bechar - nearest hotel: Hôtel Madania
siwa oasis - nearest hotel: Flower of the city hotel
chadron - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
57,tabas,IR,33.5959,56.9244,18,هتل امیر
60,holualoa,US,19.6228,-155.9522,75,Kona Hotel
67,hilo,US,19.7297,-155.0900,84,Dolphin Bay Hotel
71,qina,EG,26.1642,32.7267,36,Basma Hotel
84,at taraf,SA,25.3628,49.7250,65,No hotel found
130,hawaiian paradise park,US,19.5933,-154.9731,81,No hotel found
145,ad dilam,SA,23.9915,47.1627,25,No hotel found
184,north platte,US,41.1239,-100.7654,34,Howard Johnson Inn - North Platte
272,figuig (centre),MA,32.1089,-1.2286,44,No hotel found
301,jining,CN,35.4050,116.5814,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    color='City',
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    tiles='OSM',
    title='City Humidity Levels with Hotel Information'
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)